In [9]:
# Convolutional neural network
# Made up of neurons
# Applies convolutional filters

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [11]:
#Device, gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Hyper params
num_epochs = 20
batch_size = 4
learning_rate = 0.001

cuda


In [12]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_datasets = torchvision.datasets.CIFAR10(root='./data', train=True,transform=transforms.ToTensor(), download=True)
test_datasets = torchvision.datasets.CIFAR10(root='./data', train=False,transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_datasets, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [13]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        #Colors: 3, output: 6, kernel_size: 5
        self.conv1 = nn.Conv2d(3,6,5)
        #kernel_size: 2, stride: 2
        self.pool = nn.MaxPool2d(2, 2)
        #Input = conv1 output som er 6
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = ConvNet().to(device)

In [14]:
#Loss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(images)
        loss = criterion(outputs,labels)

        #backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if(i+1) % 2000 == 0:
            print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')

print('Finished Training')

C:\Users\Morten\anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch 1 / 4, step 2000/12500, loss = 2.3275
epoch 1 / 4, step 4000/12500, loss = 2.3059
epoch 1 / 4, step 6000/12500, loss = 2.3109
epoch 1 / 4, step 8000/12500, loss = 2.2722
epoch 1 / 4, step 10000/12500, loss = 2.2719
epoch 1 / 4, step 12000/12500, loss = 2.3024
epoch 2 / 4, step 2000/12500, loss = 2.2933
epoch 2 / 4, step 4000/12500, loss = 2.2903
epoch 2 / 4, step 6000/12500, loss = 2.1947
epoch 2 / 4, step 8000/12500, loss = 2.1826
epoch 2 / 4, step 10000/12500, loss = 2.9240
epoch 2 / 4, step 12000/12500, loss = 1.7364
epoch 3 / 4, step 2000/12500, loss = 1.4776
epoch 3 / 4, step 4000/12500, loss = 1.7571
epoch 3 / 4, step 6000/12500, loss = 2.0293
epoch 3 / 4, step 8000/12500, loss = 2.2116
epoch 3 / 4, step 10000/12500, loss = 2.2234
epoch 3 / 4, step 12000/12500, loss = 1.1848
epoch 4 / 4, step 2000/12500, loss = 1.8512
epoch 4 / 4, step 4000/12500, loss = 1.9267
epoch 4 / 4, step 6000/12500, loss = 1.5419
epoch 4 / 4, step 8000/12500, loss = 1.2023
epoch 4 / 4, step 10000/12

In [18]:
#test
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)

        _, predicted = torch.max(outputs,1)
        n_samples += labels.shape[0]
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if(label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

acc = 100.0 *n_correct/n_samples
print(f'Accuracy = {acc}')

for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc} %')


Accuracy = 39.15
Accuracy of plane: 48.0 %
Accuracy of car: 39.6 %
Accuracy of bird: 7.1 %
Accuracy of cat: 13.4 %
Accuracy of deer: 57.1 %
Accuracy of dog: 42.6 %
Accuracy of frog: 41.8 %
Accuracy of horse: 34.4 %
Accuracy of ship: 55.4 %
Accuracy of truck: 52.1 %
